### Import libs

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
import os
sys.path.append(os.path.join(sys.path[0], '..'))
import pickle
from tqdm import tqdm

from tools.corpus import PlotCorpus
from utils.doc2vec import Doc2Vec
from tools.film_card import FilmCard
from utils.metrics import mrr
from utils.inverse_index import InverseIndex
from utils.tokenizers import lemma_tokenizer

from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

import logging
logging.basicConfig(format='%(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logging.info('Doc2vec learning...')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO: Doc2vec learning...


### Data Preparing

In [2]:
with open(r'./data/dataset/films_wiki.pickle', 'rb') as f:
    films_wiki = pickle.load(f)
film_plots = [film.get_box('plot') for film in films_wiki]

In [3]:
plot_corpus = PlotCorpus(film_plots)
for sent in plot_corpus:
    print(sent)
    break

['bartender', 'working', 'saloon', 'serving', 'drink', 'customer']


### Word2Vec model 

In [4]:
w2v_model = Word2Vec(min_count=2, size=300, window=10, workers=10)
w2v_model.build_vocab(plot_corpus)
w2v_model.train(plot_corpus, total_examples=w2v_model.corpus_count, epochs=30)

INFO: collecting all words and their counts
INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO: PROGRESS: at sentence #10000, processed 99903 words, keeping 14164 word types
INFO: PROGRESS: at sentence #20000, processed 204707 words, keeping 20056 word types
INFO: PROGRESS: at sentence #30000, processed 312414 words, keeping 24812 word types
INFO: PROGRESS: at sentence #40000, processed 423257 words, keeping 28831 word types
INFO: PROGRESS: at sentence #50000, processed 533618 words, keeping 32105 word types
INFO: PROGRESS: at sentence #60000, processed 645276 words, keeping 35444 word types
INFO: PROGRESS: at sentence #70000, processed 754180 words, keeping 38310 word types
INFO: PROGRESS: at sentence #80000, processed 864046 words, keeping 41199 word types
INFO: PROGRESS: at sentence #90000, processed 975016 words, keeping 44029 word types
INFO: PROGRESS: at sentence #100000, processed 1085775 words, keeping 46894 word types
INFO: PROGRESS: at sentence #1100

INFO: EPOCH 1 - PROGRESS: at 11.25% examples, 33414 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 11.80% examples, 33215 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 12.34% examples, 33125 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 12.89% examples, 33086 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 13.16% examples, 32069 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 13.30% examples, 31235 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 13.44% examples, 30407 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 13.57% examples, 29753 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 13.72% examples, 29075 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 13.85% examples, 28417 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 13.99% examples, 27720 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 14.26% examples, 26633 words/s, in_q

INFO: EPOCH 1 - PROGRESS: at 51.13% examples, 25781 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 51.67% examples, 25866 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 52.21% examples, 25956 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 52.75% examples, 26043 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 53.28% examples, 26120 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 53.91% examples, 26237 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 54.44% examples, 26318 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 54.97% examples, 26392 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 55.50% examples, 26465 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 55.91% examples, 26451 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 56.34% examples, 26438 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 1 - PROGRESS: at 56.92% examples, 26495 words/s, in_q

INFO: EPOCH 2 - PROGRESS: at 2.38% examples, 35177 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 2.96% examples, 35813 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 3.54% examples, 35566 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 4.24% examples, 36213 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 4.80% examples, 36485 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 5.34% examples, 36468 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 5.88% examples, 36512 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 6.56% examples, 36967 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 7.25% examples, 37143 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 7.92% examples, 37202 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 8.46% examples, 37112 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 9.16% examples, 37195 words/s, in_qsize 0, out_

INFO: EPOCH 2 - PROGRESS: at 58.27% examples, 38028 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 58.79% examples, 38017 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 59.43% examples, 38032 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 60.06% examples, 38075 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 60.56% examples, 38079 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 61.21% examples, 38086 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 61.75% examples, 38089 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 62.27% examples, 38094 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 62.81% examples, 38087 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 63.36% examples, 38068 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 64.04% examples, 38094 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 2 - PROGRESS: at 64.58% examples, 38104 words/s, in_q

...

INFO: EPOCH 28 - PROGRESS: at 95.78% examples, 36659 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 28 - PROGRESS: at 96.45% examples, 36683 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 28 - PROGRESS: at 96.98% examples, 36686 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 28 - PROGRESS: at 97.60% examples, 36707 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 28 - PROGRESS: at 98.21% examples, 36731 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 28 - PROGRESS: at 98.77% examples, 36731 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 28 - PROGRESS: at 99.33% examples, 36718 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 28 - PROGRESS: at 99.86% examples, 36721 words/s, in_qsize 0, out_qsize 0
INFO: worker thread finished; awaiting finish of 9 more threads
INFO: worker thread finished; awaiting finish of 8 more threads
INFO: worker thread finished; awaiting finish of 7 more threads
INFO: worker thread finished; awaiting finish of 6 more threads
INFO: worker thread finished; awaiting finish of 5 more 

INFO: EPOCH 29 - PROGRESS: at 47.20% examples, 38448 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 47.75% examples, 38380 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 48.28% examples, 38363 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 48.86% examples, 38315 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 49.52% examples, 38340 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 50.18% examples, 38356 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 50.86% examples, 38394 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 51.41% examples, 38403 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 51.94% examples, 38382 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 52.48% examples, 38362 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 53.02% examples, 38365 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 29 - PROGRESS: at 53.54% examples, 38356 w

INFO: worker thread finished; awaiting finish of 1 more threads
INFO: worker thread finished; awaiting finish of 0 more threads
INFO: EPOCH - 29 : training on 7234947 raw words (7115753 effective words) took 189.9s, 37470 effective words/s
INFO: EPOCH 30 - PROGRESS: at 0.60% examples, 32388 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 1.21% examples, 32865 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 1.81% examples, 34731 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 2.38% examples, 35521 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 2.96% examples, 35277 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 3.54% examples, 35455 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 4.11% examples, 35654 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 4.65% examples, 36065 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 5.21% examples, 36398 words/s, in_qsize 0, out_qsi

INFO: EPOCH 30 - PROGRESS: at 50.86% examples, 37487 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 51.53% examples, 37510 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 52.08% examples, 37495 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 52.62% examples, 37490 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 53.15% examples, 37501 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 53.67% examples, 37518 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 54.18% examples, 37524 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 54.85% examples, 37549 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 55.35% examples, 37554 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 55.91% examples, 37557 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 56.49% examples, 37549 words/s, in_qsize 0, out_qsize 0
INFO: EPOCH 30 - PROGRESS: at 57.05% examples, 37531 w

(213472460, 217048410)

### TFIDF weights model

In [5]:
dct = Dictionary(plot_corpus)  # fit dictionary
model = TfidfModel(dictionary=dct)  # fit model

INFO: adding document #0 to Dictionary(0 unique tokens: [])
INFO: adding document #10000 to Dictionary(14164 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...)
INFO: adding document #20000 to Dictionary(20056 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...)
INFO: adding document #30000 to Dictionary(24812 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...)
INFO: adding document #40000 to Dictionary(28831 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...)
INFO: adding document #50000 to Dictionary(32105 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...)
INFO: adding document #60000 to Dictionary(35444 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...)
INFO: adding document #70000 to Dictionary(38310 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...)
INFO: adding document #80000 to Dictionary(41199 unique tokens: ['barten

INFO: built Dictionary(165839 unique tokens: ['bartender', 'customer', 'drink', 'saloon', 'serving']...) from 659344 documents (total 7234947 corpus positions)


### Doc2Vec model

In [6]:
d2v = Doc2Vec(w2v_model, tfidf_model=model, tokenizer=lemma_tokenizer)
len(d2v.get_sentence_vector(next(iter(plot_corpus)), True))

300

In [7]:
d2v_mean = Doc2Vec(w2v_model, tokenizer=lemma_tokenizer)
len(d2v_mean.get_sentence_vector(next(iter(plot_corpus)), True))

300

### Save doc2vec

In [8]:
basedir = r'./data'
if not os.path.exists(basedir):
    os.makedirs(basedir)
    
with open(r'./data/doc2vec.model', 'wb') as f:
    pickle.dump(d2v, f)
    
with open(r'./data/doc2vec_mean.model', 'wb') as f:
    pickle.dump(d2v_mean, f)

In [9]:
with open(r'./data/doc2vec.model', 'rb') as f:
    d2v2 = pickle.load(f)

In [10]:
len(d2v2.get_sentence_vector(next(iter(plot_corpus)), True))

300